<h2>k-Nearest Neighbor (kNN) Classification</h2>
<p>The kNN classifier consists of two stages:</p>
<ul>
<li> During training, the classifier takes the training data and simply remembers it.</li>
<li> During testing, kNN classifies every test image by comparing to all training images and transferring the labels of the k most similar examples .</li>
<li> The valur of k is cross-validated</li>
</ul>

### As for the dataset:
<ul>
<li> It contains 28709 training data points.</li>
<li> And 7178 testing data points</li>
<p>The data consists of 48x48 pixel grayscale images of faces. The faces have been automatically registered so that the face is more or less centered and occupies about the same amount of space in each image. The task is to categorize each face based on the emotion shown in the facial expression in to one of seven categories: `{0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral}`.<br>
<b>X_train.csv</b> contains 28709 rows(examples) and 2304(i.e 48 x 48) columns(columns represent the pixel values). <b>X_test.csv</b> contains 7178 rows and 2304 columns. <b>y_train.csv</b> & <b>y_test.csv</b> contains the numeric code ranging from 0 to 6 representing a certain emotion.</p>
</ul>

In [ ]:
# import some important packages
from time import time
import random
import numpy as np
import matplotlib.pyplot as plt
from k_nearest_neighbor import KNearestNeighbor

# matplotlib figures appear inline in the notebook rather than a new window
%matplotlib inline
plt.rcParams["figure.figsize"] = (10.0, 8.0)     # set default size of plots
plt.rcParams["image.interpolation"] = "kaiser"
plt.rcParams["image.cmap"] = "gray"

%load_ext autoreload
%autoreload 2

In [ ]:
# Load the dataset
X_train = np.genfromtxt("datasets/X_train.csv", delimiter = ",")
y_train = np.genfromtxt("datasets/y_train.csv", delimiter = ",")
X_test = np.genfromtxt("datasets/X_test.csv", delimiter = ",")
y_test = np.genfromtxt("datasets/y_test.csv", delimiter = ",")

In [ ]:
# for sanity check, print out the size of the training and test data
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

In [ ]:
# Visualize some examples from the dataset
# Show a few exapmles of training images from each class
classes = ["angry", "disgust", "fear", "happy", "sad", "surprise", "neutral"]
num_classes = len(classes)
samples_per_class = 6
for y, cls in enumerate(classes):
    idxs = np.flatnonzero(y_train == y)
    idxs = np.random.choice(idxs, samples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        plt.imshow(np.reshape(X_train[idx], (48, 48)))
        plt.axis('off')
        if i == 0:
            plt.title(cls)
plt.show()

In [ ]:
# Subsample the data for more efficient code execution
num_training = 5000
mask = list(range(num_training))
Xtrain = X_train[mask]
ytrain = y_train[mask]

num_test = 500
mask = list(range(num_test))
Xtest = X_test[mask]
ytest = y_test[mask]
print(Xtrain.shape, Xtest.shape)

### Now we will be using the kNN Classifier on the subsamples of the whole dataset

In [ ]:
# Create a kNN classifier instance
# and train
classifier = KNearestNeighbor()
classifier.train(Xtrain, ytrain)

<p>Now we would classify the test data with the kNN classifier. Recall that we can break down this step into two steps:</p>
<ol>
<li>First compute distances between all test examples and all training examples.</li>
<li>Given these distances, for each test example, find the k nearest examples and have them vote for the label.</li>
</ol>
<p>Lets start with computing the distance matrix between all training and testing examples. For example, if there are <b>Ntr</b> training examples and <b>Nte</b> testing examples, this step should result in <b>Nte x Ntr</b> matrix where each element `(i, j)` is the `distance between i-th test example and j-th training example.`</p>

In [ ]:
# used time() to get to know about the time elapsed
# no vectorization at all to speed up in compute_distances_two_loops(). So this is too slow
start = time()
distance_matrix = classifier.compute_distances_two_loops(Xtest)
end = time()
time_taken = end - start
print("Time taken: {}".format(time_taken))
print(distance_matrix.shape)

In [ ]:
# To visualize the distance matrix: each row is a single test example and its distances to training examples
plt.figure(figsize = (18.0, 18.0))
plt.imshow(distance_matrix, interpolation = "nearest") #kaiser would make it smooth
# and we wont be able to visualize the pattern
plt.show()
# Black indicates low distances while white indicates high distances

In [ ]:
# Implementation of the function predict_labels.
# using k = 1 (1 nearest neighbour)
# Also print out the accuracy at particular value of k
y_test_pred = classifier.predict_labels(distance_matrix, k = 1)
# Compute and print function of corretly predicted labels
num_correct = np.sum(y_test_pred == ytest)
accuracy = float(num_correct / num_test)
print("Predicted %d / %d correct -> accuracy: %f"%(num_correct, num_test, accuracy))

In [ ]:
# Now change the value of k to 5
y_test_pred = classifier.predict_labels(distance_matrix, k = 5)
# Compute and print function of corretly predicted labels
num_correct = np.sum(y_test_pred == ytest)
accuracy = float(num_correct / num_test)
print("Predicted %d / %d correct -> accuracy: %f"%(num_correct, num_test, accuracy))

`We can observe a slightly bad performance when k = 5. We need some sort of algorithm to find the best value of k (will be done below).`

In [ ]:
# Now lets speed up distance matrix by using partial vectorization with one loop.
# Implement the function compute_distances_one_loops
start = time()
distance_matrix_one = classifier.compute_distances_one_loop(Xtest)
end = time()
time_taken = end - start
print("Time taken: {}".format(time_taken))
# There are many ways to check whether two matrices are similar; 
# one of the simplest is the Forbenius Norm . 
# The Frobenius norm of two matrices is the square root of the squared sum of differences of all elements.
# Simply put you need to reshape the matrices into vectors and compute the Euclidean distance between them.
difference = np.linalg.norm(distance_matrix - distance_matrix_one, ord = 'fro')
print('Difference was: %f' % (difference, ))
if difference < 0.001:
    print('Good! The distance matrices are the same')
else:
    print('Uh-oh! The distance matrices are different')

In [ ]:
# Now lets implement the FULLY VECTORIZED one
start = time()
distance_matrix_two = classifier.compute_distances_no_loops(Xtest)
end = time()
print("Time Taken: {}".format(end - start))
difference = np.linalg.norm(distance_matrix - distance_matrix_two, ord = 'fro')
print('Difference was: %f' % (difference, ))
if difference < 0.001:
    print('Good! The distance matrices are the same')
else:
    print('Uh-oh! The distance matrices are different')

`We can totally notice the change in excution time. Fully vectorized one was the best one.`

### Cross-Validation

We have implemented the k-Nearest Neighbor classifier but we set the value k = 5 arbitrarily (although it made the accuracy worse). We will now determine the best value of this hyperparameter with cross-validation.

In [ ]:
num_folds = 5
k_choices = [1, 3, 5, 8, 10, 12, 15, 20, 50, 100]

X_train_folds = []
y_train_folds = []
                                                                        
# Spliting up the training data into folds. After splitting, X_train_folds and
# y_train_folds should each be lists of length num_folds, where          
# y_train_folds[i] is the label vector for the points in X_train_folds[i].

X_train_folds = np.array_split(Xtrain, num_folds)
y_train_folds = np.array_split(ytrain, num_folds)


# A dictionary holding the accuracies for different values of k that we find
# when running cross-validation.
k_to_accuracies = {}


# Perform k-fold cross validation to find the best value of k. For each       
# possible value of k, run the k-nearest-neighbor algorithm num_folds times.                              
for k in k_choices:
    for fold in range(num_folds):
        valid_X_test = X_train_folds[fold]
        valid_y_test = y_train_folds[fold]
        fold_X_train = np.concatenate(X_train_folds[ : fold] + X_train_folds[fold + 1 : ])
        fold_y_train = np.concatenate(y_train_folds[ : fold] + y_train_folds[fold + 1 : ])
        
        # create instance of kNN classifier and train it
        a_classifier = KNearestNeighbor()
        a_classifier.train(fold_X_train, fold_y_train)
        
        # Computing Distances
        distance_for_folds = a_classifier.compute_distances_no_loops(valid_X_test)
        fold_y_test_predict = a_classifier.predict_labels(distance_for_folds, k = k)
        
        # Now check accuracy
        fold_num_correct = np.sum(fold_y_test_predict == valid_y_test)
        fold_num_test = valid_X_test.shape[0]
        fold_accuracy = float(fold_num_correct) / fold_num_test
        
        # Writing in the dictionary k_to_accuracies
        k_to_accuracies[k] = k_to_accuracies.get(k, []) + [fold_accuracy]         

# Print out the computed accuracies
for k in sorted(k_to_accuracies):
    for accuracy in k_to_accuracies[k]:
        print('k = %d, accuracy = %f' % (k, accuracy))

`The accuracy is Heighest i.e 30.1% when k = 50. So we will be using that to predict.`

### A representation of how cross-validation accuracy varied with k

In [ ]:
# plot the raw observations
for k in k_choices:
    accuracies = k_to_accuracies[k]
    plt.scatter([k] * len(accuracies), accuracies)

# plot the trend line with error bars that correspond to standard deviation
accuracies_mean = np.array([np.mean(v) for k,v in sorted(k_to_accuracies.items())])
accuracies_std = np.array([np.std(v) for k,v in sorted(k_to_accuracies.items())])
plt.errorbar(k_choices, accuracies_mean, yerr=accuracies_std)
plt.title('Cross-validation on k')
plt.xlabel('k')
plt.ylabel('Cross-validation accuracy')
plt.show()

In [ ]:
# Based on the cross-validation results above, choose the best value for k,   
# retrain the classifier using all the training data, and test it on the test data
best_k = 50

classifier = KNearestNeighbor()
classifier.train(Xtrain, ytrain)
y_test_pred = classifier.predict(Xtest, k=best_k)

# Compute and display the accuracy
num_correct = np.sum(y_test_pred == ytest)
accuracy = float(num_correct / num_test)
print('Got %d / %d correct => accuracy: %f' % (num_correct, num_test, accuracy))

<h3>---------------------------------------------------------------------------------------------------------------------------------------------------------</h3>

### Now Lets use the whole dataset

In [ ]:
# sanity check that we didnt mess up the actual dataset
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
# instance of kNN for the whole dataset
dataset_classifier = KNearestNeighbor()
dataset_classifier.train(X_train, y_train)

In [ ]:
# using the fully vectorized function to reduce the time taken.
# also printing out the time taken, to run on the whole dataset
start = time()
dataset_classifier.compute_distances_no_loops(X_test)
end = time()
print("Time Taken: {}".format(end - start))

In [ ]:
# Compute and display the accuracy
# and also compute the prediction time
start = time()
dataset_y_test_pred = dataset_classifier.predict(X_test, k = 50)
end = time()
dataset_num_correct = np.sum(dataset_y_test_pred == y_test)
dataset_num_test = X_test.shape[0]
dataset_accuracy = float(dataset_num_correct / dataset_num_test)

In [ ]:
print("Time taken: {}".format(end - start))
print('Got %d / %d correct => accuracy: %f' % (dataset_num_correct, dataset_num_test, dataset_accuracy))

### So we acheived a total of 31.27% accuracy on using the whole dataset, which is pretty good, because we used a very naive ML Algorithm.